# Exploring the results
## Output files
### summary_statistics.txt
### gene_presence_absence.csv
### gene_presence_absence.Rtab
### pan_genome_reference.fa
### *.Rtab
### accessory_binary_genes.fa.newick
### accessory_graph.dot
### core_accessory_graph.dot
### clustered_proteins
### core_gene_alignment.aln
## query_pan_genome
## roary_plots.py
## Draw a tree form the core gene alignment
FastTree –nt –gtr core_gene_alignment.aln > my_tree.newick